### OVERALL LOOK AT THE DATASET AND SILVER LAYER

In [ ]:
import pandas as pd
import numpy as np
import pymongo
import re
from jproperties import Properties
from sqlalchemy import create_engine, text
from sqlalchemy import Numeric
import subprocess


# pandas settings
pd.set_option('max.colwidth', 200)
pd.set_option('display.max_rows', 132)

In [ ]:
# loading variables from file (db access)

with open("app.properties", "r+b") as f:
    p = Properties()
    p.load(f, "utf-8")

### Mongodb Acess (Data Source)

In [3]:

client = pymongo.MongoClient()
db = client.get_database('zap_imoveis')
bronze_layer = db.get_collection('bronze')

In [4]:
print(f'Number of data records in the database: {bronze_layer.count_documents({})}')

Number of data records in the database: 20038


### Creating a DataFrame


In [5]:

df = pd.DataFrame(list(bronze_layer.find({})))

assert df[df['data-id'].duplicated()]['data-id'].sum() == 0, 'We have duplicated ids (data-id) in the DB'

In [6]:
df.head(2)

,_id,link,data-id,atts,date
0,67a344c52b111f021af8a118,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-2-quartos-com-piscina-saude-zona-sul-sao-paulo-sp-112m2-id-2774950239/,2774950239,"{'ad_type': 'Aluguel', 'sell_price': None, 'rent_price': 'R$ 5.000/mês', 'condo_price': 'R$ 970', 'address': 'Rua Doutor Samuel Porto, 237 - Saúde, São Paulo - SP', 'rstate_type': 'Apartamentos pa...",20250224
1,67a344cc2b111f021af8a119,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-1-quarto-com-piscina-cambuci-zona-sul-sao-paulo-sp-37m2-id-2733414161/,2733414161,"{'ad_type': 'Aluguel', 'sell_price': None, 'rent_price': 'R$ 2.400/mês', 'condo_price': 'R$ 980', 'address': 'Rua Backer, 338 - Cambuci, São Paulo - SP', 'rstate_type': 'Apartamentos para Alugar/'...",20250224


### Getting Only The Attributes  

In [7]:
df_atts =  pd.concat(
        [df[['data-id','link']], df['atts'].apply(pd.Series)],
        axis = 1
)

In [8]:
df_atts.head(2)

,data-id,link,ad_type,sell_price,rent_price,condo_price,address,rstate_type,floorSize,numberOfRooms,...,BEAUTY_ROOM,VINYL_FLOOR,DRESS_ROOM,CORRAL,DRYWALL,GOLF_FIELD,SMART_CONDOMINIUM,ECO_CONDOMINIUM,DIVIDERS,CARPET
0,2774950239,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-2-quartos-com-piscina-saude-zona-sul-sao-paulo-sp-112m2-id-2774950239/,Aluguel,None,R$ 5.000/mês,R$ 970,"Rua Doutor Samuel Porto, 237 - Saúde, São Paulo - SP",Apartamentos para Alugar/,112 m²,2 quartos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2733414161,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-1-quarto-com-piscina-cambuci-zona-sul-sao-paulo-sp-37m2-id-2733414161/,Aluguel,None,R$ 2.400/mês,R$ 980,"Rua Backer, 338 - Cambuci, São Paulo - SP",Apartamentos para Alugar/,37 m²,1 quarto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
## removing comercial as we are not interested with the scope of this project;

df_atts = df_atts[~df_atts.rstate_type.apply(lambda x: True if re.search('Comerciais', str(x)) else False)]
print(df_atts.rstate_type.unique())

['Apartamentos para Alugar/' 'Flats para Alugar/' 'Imóveis para Alugar/'
 'Casas para Alugar/' 'Apartamentos à Venda/' 'Coberturas à Venda/'
 'Sobrados à Venda/' 'Casas de Condomínio para Alugar/'
 'Sobrados para Alugar/' 'Studios para Alugar/'
 'Casas de Condomínio à Venda/' 'Coberturas para Alugar/' 'Casas à Venda/'
 'Casas de Vila à Venda/' 'Kitnets para Alugar/' 'Flats à Venda/'
 'Studios à Venda/' 'Casas de Vila para Alugar/' nan 'Lofts para Alugar/'
 'Imóveis à Venda/' 'Fazendas, Sítios e Chácaras para Alugar/'
 'Fazendas, Sítios e Chácaras à Venda/' 'Lofts à Venda/']


### Selecting Columns Of Interest

In [10]:
# for _ in df_atts.columns:
#      print(_)

cols_of_interest = [
    'data-id',
    'ad_type', 'rstate_type',
    'rent_price', 'condo_price', 'sell_price',
    'address', 'floorSize', 'numberOfBathroomsTotal',
    'numberOfParkingSpaces',
    'numberOfRooms',
    'numberOfSuites',
    'FURNISHED',
    'AIR_CONDITIONING',
    'HOME_OFFICE',
    'POOL',
    'GYM',
    'SAUNA',
    'LAUNDRY',
    'COWORKING',
    'link'
    ]

# keep only the cols that exist in the dataset; 

cols_of_interest = [x for x in cols_of_interest if x in df_atts.columns]

# change the df
df_atts = df_atts[cols_of_interest]


### % of NaN Values

In [11]:
missing = df_atts.isna().sum()
missing = missing/df_atts.shape[0] * 100
missing.sort_values(ascending=False)

## comments; 
# the null values on the ameneties represents the non presence of such elements like GYM or LAUNDRY.
# for others that are mandatory (like at least sell or rent price and size), its more likely to be an error in the webscrap phase. 

COWORKING                 99.310896
SAUNA                     91.381204
HOME_OFFICE               89.683412
GYM                       88.989314
LAUNDRY                   88.090482
FURNISHED                 83.626286
AIR_CONDITIONING          80.575252
POOL                      73.194847
sell_price                66.992909
numberOfSuites            20.013982
numberOfParkingSpaces      7.814841
numberOfRooms              1.338260
numberOfBathroomsTotal     0.988715
floorSize                  0.978728
rent_price                 0.369520
ad_type                    0.279636
rstate_type                0.279636
address                    0.279636
condo_price                0.279636
data-id                    0.000000
link                       0.000000
dtype: float64

### Removing Null Values
<li> if m2 = 0/NA or both rent and sell price are 0/NA, we'll ignore (this could be due an error in the webscraping phase)


In [12]:
df_atts.shape

(20026, 21)

In [13]:
df_atts[
    (df_atts.rent_price.isnull()) &
    (df_atts.sell_price.isnull())
    ]

,data-id,ad_type,rstate_type,rent_price,condo_price,sell_price,address,floorSize,numberOfBathroomsTotal,numberOfParkingSpaces,...,numberOfSuites,FURNISHED,AIR_CONDITIONING,HOME_OFFICE,POOL,GYM,SAUNA,LAUNDRY,COWORKING,link
2942,2711673125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-de-condominio-4-quartos-com-churrasqueira-alto-da-boa-vista-sao-paulo-442m2-id-2711673125/
3103,2762346056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/venda-sobrados-4-quartos-com-cozinha-planalto-paulista-sao-paulo-600m2-id-2762346056/
5316,2771360724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-studio-1-quarto-mobiliado-vila-mariana-sao-paulo-25m2-id-2771360724/
5698,2705779417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/venda-casa-6-quartos-com-cozinha-jardim-america-sao-paulo-701m2-id-2705779417/
6110,2776713149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-3-quartos-com-churrasqueira-vila-santo-estevao-sao-paulo-85m2-id-2776713149/
6697,2652471358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/venda-casa-4-quartos-com-piscina-jardim-luzitania-sao-paulo-753m2-id-2652471358/
6828,2764867732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/venda-casa-de-condominio-4-quartos-mobiliado-campo-belo-sao-paulo-700m2-id-2764867732/
6878,2774214915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-4-quartos-com-piscina-brooklin-paulista-sao-paulo-690m2-id-2774214915/
6879,2774256835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-4-quartos-com-piscina-brooklin-paulista-sao-paulo-690m2-id-2774256835/
6881,2774200216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zapimoveis.com.br/imovel/aluguel-casa-4-quartos-com-piscina-brooklin-paulista-sao-paulo-690m2-id-2774200216/


In [14]:
# main reason for this remove is because the mainly focus here is in the price and size. So, both prices as NA or the size as NA makes the instance useless

df_atts = df_atts[~df_atts.floorSize.isnull()]

df_atts = df_atts[
    ~(df_atts.rent_price.isnull()) |
    ~(df_atts.sell_price.isnull())
    ]

In [15]:
df_atts.shape

(19830, 21)

### Data Transformation 

<li> Dtypes


In [16]:
# FLOAT 

def func(x):
    p = re.compile('[\d\.\,\- ]+')

    match = p.search(str(x))

    if match:
        return match.group(0).replace('.', '')
    else:
        return x



cols_float = ['sell_price', 'rent_price', 'condo_price']

df_atts = df_atts.replace({'isento': np.nan,'não informado': np.nan})

df_atts[cols_float] = df_atts[cols_float].map(func).replace('.', '')

df_atts[cols_float] = df_atts[cols_float].astype(float)
      

<li> Boolean


In [17]:

cols_bool = ['AIR_CONDITIONING', 'FURNISHED','HOME_OFFICE', 'POOL','GYM', 'LAUNDRY', 'SAUNA', 'COWORKING'] # desired;

cols_bool = [col for col in cols_bool if col in df_atts.columns] # ajusted in relation to the dataset;



df_atts.loc[:, cols_bool] = df_atts[cols_bool].fillna(0) # 0 represents the unpresence of such property; 
df_atts.loc[:, cols_bool]= df_atts[cols_bool].replace(to_replace= ('^(?!0).*'), value = 1, regex= True) # all the rest can be replaced by 1 (True)

/tmp/ipykernel_12025/2694258841.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_atts.loc[:, cols_bool]= df_atts[cols_bool].replace(to_replace= ('^(?!0).*'), value = 1, regex= True) # all the rest can be replaced by 1 (True)


<li> Integer


In [18]:

cols_int = ['floorSize', 'numberOfBathroomsTotal', 'numberOfParkingSpaces', 'numberOfRooms'] + cols_bool
df_atts[cols_int] = df_atts[cols_int].map(func)
df_atts[cols_int] = df_atts[cols_int].fillna(0).astype(int)

### Feature Engineering


In [19]:
def regex_neighbor(x):

    match=re.search('(?<=- )[\w ]+', str(x))
    if match:
        return match.group()
    else:
        return x

df_atts.loc[:, 'neighborhood'] = df_atts.address.apply(regex_neighbor)
df_atts.loc[:, 'numberOfSuites'] = df_atts.numberOfSuites.fillna(0).apply(lambda x: re.search('\d+', str(x)).group())
df_atts['total_rental_price'] = df_atts.condo_price + df_atts.rent_price

# to int;
df_atts['numberOfSuites'] = df_atts.numberOfSuites.astype(int)

In [20]:
for index, cols in df_atts.iterrows():
    
    if not isinstance(cols.address, float):
        address = cols.address.split(',')
        
        n = len(address)

        df_atts.loc[index, 'neighborhood']  = regex_neighbor(address[-2])

        if n == 3: # means we have street name;

            df_atts.loc[index, 'street'] = address[-3]
            df_atts.loc[index, 'number'] = address[-2].split('-')[0].strip()

  
df_atts = df_atts.drop(columns = ['address']) # we can drop; 

In [21]:
df_atts.dtypes

data-id                    object
ad_type                    object
rstate_type                object
rent_price                float64
condo_price               float64
sell_price                float64
floorSize                   int64
numberOfBathroomsTotal      int64
numberOfParkingSpaces       int64
numberOfRooms               int64
numberOfSuites              int64
FURNISHED                   int64
AIR_CONDITIONING            int64
HOME_OFFICE                 int64
POOL                        int64
GYM                         int64
SAUNA                       int64
LAUNDRY                     int64
COWORKING                   int64
link                       object
neighborhood               object
total_rental_price        float64
street                     object
number                     object
dtype: object

### Data Cleaning

In [22]:
df_atts.loc[:, 'rstate_type'] = df_atts.rstate_type.str[:-1]  # removing '/' at final of each string;


In [23]:


cols_reord = [
             'data-id', 'rstate_type', 'ad_type',
             'rent_price',  'sell_price',  'condo_price', 'total_rental_price', 'floorSize',
             'neighborhood', 'street', 'number'] + cols_bool + \
            ['numberOfRooms', 'numberOfSuites', 'numberOfBathroomsTotal', 'numberOfParkingSpaces', 'link']

df_atts = df_atts[cols_reord]


In [24]:
renamed_cols = {'data-id': 'id', 
                'rstate_type': 'kind', 'ad_type': 'rent_or_selling', 
                'rent_price':'price_rent', 'sell_price': 'price_sale', 'condo_price': 'price_condominium', 'total_rental_price': 'rent_plus_condo', 
                'floorSize': 'floor_size', 'number': 'address_number',  
                'AIR_CONDITIONING': 'air_conditioning', 'FURNISHED': 'furnished', 'HOME_OFFICE': 'home_office', 
                "POOL": 'pool', "GYM": 'gym', "LAUNDRY": 'laundry', 'SAUNA': 'sauna',
                'numberOfRooms': 'rooms', 'numberOfSuites': 'suites', 'numberOfBathroomsTotal': 'bathrooms', 'numberOfParkingSpaces': 'parking'}


df_atts = df_atts.rename(columns = renamed_cols)

In [25]:
df_atts.head(2)

,id,kind,rent_or_selling,price_rent,price_sale,price_condominium,rent_plus_condo,floor_size,neighborhood,street,...,pool,gym,laundry,sauna,COWORKING,rooms,suites,bathrooms,parking,link
0,2774950239,Apartamentos para Alugar,Aluguel,5000.0,NaN,970.0,5970.0,112,Saúde,Rua Doutor Samuel Porto,...,1,0,0,0,0,2,1,3,3,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-2-quartos-com-piscina-saude-zona-sul-sao-paulo-sp-112m2-id-2774950239/
1,2733414161,Apartamentos para Alugar,Aluguel,2400.0,NaN,980.0,3380.0,37,Cambuci,Rua Backer,...,1,0,0,0,0,1,1,1,1,https://www.zapimoveis.com.br/imovel/aluguel-apartamento-1-quarto-com-piscina-cambuci-zona-sul-sao-paulo-sp-37m2-id-2733414161/


### Validating Duplicated Listing

In [26]:
# this happens because the same house can be annouce by different  real estate agencies


columns = list(df_atts.columns)
for i in ['id', 'link']:
    columns.remove(i)

print(f'Number of duplicated listing based on REstate properties: {df_atts.duplicated(subset= columns).sum()}')

Number of duplicated listing based on REstate properties: 217


In [27]:
df_atts[df_atts.duplicated(subset= columns, keep =False)].sort_values('street').head(4)
# notice that the id changes but its the same real estate.

,id,kind,rent_or_selling,price_rent,price_sale,price_condominium,rent_plus_condo,floor_size,neighborhood,street,...,pool,gym,laundry,sauna,COWORKING,rooms,suites,bathrooms,parking,link
6226,2778222245,Flats para Alugar,Aluguel,3000.0,NaN,1200.0,4200.0,44,Jardins,Alameda Joaquim Eugênio de Lima,...,0,0,0,0,0,1,1,1,1,https://www.zapimoveis.com.br/imovel/aluguel-flat-1-quarto-mobiliado-jardim-paulista-sao-paulo-44m2-id-2778222245/
6229,2778222524,Flats para Alugar,Aluguel,3000.0,NaN,1200.0,4200.0,44,Jardins,Alameda Joaquim Eugênio de Lima,...,0,0,0,0,0,1,1,1,1,https://www.zapimoveis.com.br/imovel/aluguel-flat-1-quarto-mobiliado-jardim-paulista-sao-paulo-44m2-id-2778222524/
13538,2779096032,Coberturas à Venda,Venda,35000.0,5850000.0,3475.0,38475.0,197,Cerqueira César,Alameda Ministro Rocha Azevedo,...,1,1,0,1,0,2,2,4,2,https://www.zapimoveis.com.br/imovel/venda-cobertura-2-quartos-com-piscina-cerqueira-cesar-sao-paulo-197m2-id-2779096032/
13536,2779097166,Coberturas à Venda,Venda,35000.0,5850000.0,3475.0,38475.0,197,Cerqueira César,Alameda Ministro Rocha Azevedo,...,1,1,0,1,0,2,2,4,2,https://www.zapimoveis.com.br/imovel/venda-cobertura-2-quartos-com-piscina-cerqueira-cesar-sao-paulo-197m2-id-2779097166/


In [28]:
df_atts = df_atts.drop_duplicates(subset=columns)

df_atts.shape

(19613, 24)

### Estatistical Cleaning

<li> outliers

In [30]:
df_atts_sell = df_atts[~df_atts.price_sale.isna()]
df_atts_rent = df_atts[~df_atts.price_rent.isna()]  # as some adtype listed as "For Sale" may contain the price and be able to rent, we can use .isna()

In [31]:
def plot_box_plot(df, sell_or_rent):

    if sell_or_rent == 'rent':
        box_data = df.price_rent / df.floor_size

    elif sell_or_rent == 'sell':
        box_data = df.price_sale / df.floor_size
    
    else:
        raise NameError

    Q1 = np.percentile(box_data, 25)
    Q3 = np.percentile(box_data, 75)
    IQR = Q3 - Q1

    u_bound = Q3 + 1.5*IQR
    l_bound = Q1 - 1.5*IQR

    return {'u_bound': u_bound, 'l_bound': l_bound}

In [32]:
# IQR to remove outliers based on price/floor_size

rent_bounds = plot_box_plot(df_atts_rent, 'rent')
df_atts_rent_adjusted = df_atts_rent[
    ((df_atts_rent.price_rent / df_atts_rent.floor_size) < rent_bounds.get('u_bound')) &
   ( (df_atts_rent.price_rent / df_atts_rent.floor_size) > rent_bounds.get('l_bound') )
]

sale_bounds = plot_box_plot(df_atts_sell, 'sell')
df_atts_sale_adjusted = df_atts_sell[
    ((df_atts_sell.price_rent / df_atts_sell.floor_size) < sale_bounds.get('u_bound')) &
   ( (df_atts_sell.price_rent / df_atts_sell.floor_size) > sale_bounds.get('l_bound') )
]


In [33]:
# making a new index to slice the original dataframe;
new_index = np.concatenate((df_atts_rent_adjusted.index,df_atts_sale_adjusted.index))
new_index = list(set(new_index))

df_atts = df_atts.loc[new_index, :]

### Postgres Ingestion;


In [ ]:
# db credentials come from the file app.properties

uri = p.get('uri').data
engine = create_engine(uri)

with engine.connect() as con:
    con.execute(text("CREATE SCHEMA if not exists zap"))
    con.commit()

In [ ]:
# drop views to update silver table; 
with engine.connect() as conn:
    views_list = conn.execute(
    text("""select table_name 
            from information_schema.views
            where table_schema = 'zap' """)).fetchall() # getting the view's name that exists;


    for view in views_list:  # dropping them because it's needed;
        conn.execute(text(f"DROP VIEW zap.{view[0]}"))
        conn.commit() 


df_atts.to_sql('silver_zapimoveis', schema='zap', # now its gonna work;
    con = uri, if_exists='replace', index = False, 
    dtype={'price_rent': Numeric, 'price_sale': Numeric, 'price_condominium': Numeric, 'rent_plus_condo': Numeric})


In [ ]:
### call run dbt;
subprocess.run("dbt run", shell=True)

01:56:05  Running with dbt=1.9.2
01:56:05  Registered adapter: postgres=1.9.0
01:56:05  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 2 unused configuration paths:
- models.mart.gold_flats_avg_prices
- models.mart.gold_apartment_avg_prices
01:56:06  Found 4 models, 1 source, 431 macros
01:56:06  
01:56:06  Concurrency: 4 threads (target='dev')
01:56:06  
01:56:06  1 of 4 START sql view model zap.gold_apartment_avg_prices ...................... [RUN]
01:56:06  2 of 4 START sql view model zap.gold_flat_kit_studio_avg_prices ................ [RUN]
01:56:06  3 of 4 START sql view model zap.gold_floorsize_ranked_avg_prices ............... [RUN]
01:56:06  4 of 4 START sql view model zap.gold_house_avg_prices .......................... [RUN]
01:56:06  2 of 4 OK created sql view model zap.gold_flat_kit_studio_avg_prices ........... [CREATE VIEW in 0.43s]
01:56:06  4 of 4 OK created sql view model zap.gold_house_avg_prices .....

CompletedProcess(args='dbt run', returncode=0)

### END